In [1]:
import commonutils

rootdir = "../datasets/ML_data/W4-11"
labelsfilename = "../datasets/ML_data/W4-11/labels.txt"
howmanydifs = 3

allvalues, basic_pbedescriptors, basic_hfdescriptors =\
      commonutils.read_dataset(rootdir, labelsfilename, howmanydifs)
